# DEAP 的基本使用

本文主要参考了：

- [DEAP documentation](https://deap.readthedocs.io/en/master/)
- [基于DEAP库的Python进化算法从入门到入土](https://www.jianshu.com/p/8fa044ed9267)

进化算法(Evolutionary Algorithms)是一类元启发式算法的统称。这类算法借鉴大自然中生物的进化、选择与淘汰机制，通常先产生一个族群，然后不断进化与淘汰，最终产生能够在严酷的自然环境中生存的优异个体（也就是有较大适应度函数的可行解）。它具有自组织、自适应的特性，常被用来处理传统优化算法难以解决的问题。

其优点在于：

- 泛用性强，对连续变量和离散变量都能适用；
- 不需要导数信息，因此不要求适应度函数的连续和可微性质(或者说不需要问题内在机理的相关信息)；
- 可以在解空间内大范围并行搜索；
- 不容易陷入局部最优；
- 高度并行化，并且容易与其他优化方法整合。

缺点包括：

- 对于凸优化问题，相对基于梯度的优化方法（例如梯度下降法，牛顿/拟牛顿法）收敛速度更慢；
- 进化算法需要在搜索空间投放大量个体来搜索最优解。对于高维问题，由于搜索空间随维度指数级膨胀，需要投放的个体数也大幅增长，会导致收敛速度变慢；
- 设计编码方式、适应度函数以及变异规则需要大量经验。

宽泛来讲，大部分进化算法都具有以下元素：

- 个体编码(Individual representation): 将问题的解空间编码映射到搜索空间的过程。常用的编码方式有二值编码(Binary)，格雷编码(Gray)，浮点编码(Floating-point)等。
- 评价(Evaluation): 设定一定的准则评价族群内每个个体的优秀程度。这种优秀程度通常称为适应度(Fitness)。
- 配种选择(Mating selection): 建立准则从父代中选择个体参与育种。尽可能选择精英个体的同时也应当维护种群的多样性，避免算法过早陷入局部最优。
- 变异(Variation): 变异过程包括一系列受到生物启发的操作，例如重组(Recombination)，突变(mutation)等。通过变异操作，父代的个体编码以一定方式继承和重新组合后，形成后代族群。
- 环境选择(Environmental selection): 将父代与子代重组成新的族群。这个过程中育种得到的后代被重新插入到父代种群中，替换父代种群的部分或全体，形成具有与前代相近规模的新族群。
- 停止准则(Stopping crieterion): 确定算法何时停止，通常有两种情况：算法已经找到最优解或者算法已经选入局部最优，不能继续在解空间内搜索。

利用这些元素，我们就可以依照流程图组成一个进化算法：

![](pictures/17867674-a70e9d4134cdbad6.webp)

文字表述如下：

``` pseudo-code
Generate the initial population P(0) at random, and set t to 0.
repeat
    Evaluate the fitness of each individual in P(t).
    Select parents from P(t) based on their fitness.
    Obtain population P(t+1) by making variations to parents.
    Set t = t + 1
until Stopping crieterion satisfied
```

下面就看看在DEAP中如何执行这些步骤。

安装deap执行以下语句即可：

```Shell
conda install -c conda-forge deap
```

## 优化问题的定义

- 单目标优化：creator.create('FitnessMin', base.Fitness, weights=(-1.0, ))

在创建单目标优化问题时，weights用来指示最大化和最小化。此处-1.0即代表问题是一个最小化问题，对于最大化，应将weights改为正数，如1.0。

另外即使是单目标优化，weights也需要是一个tuple，以保证单目标和多目标优化时数据结构的统一。

对于单目标优化问题，weights 的绝对值没有意义，只要符号选择正确即可。

- 多目标优化：creator.create('FitnessMulti', base.Fitness, weights=(-1.0, 1.0))

对于多目标优化问题，weights用来指示多个优化目标之间的相对重要程度以及最大化最小化。如示例中给出的(-1.0, 1.0)代表对第一个目标函数取最小值，对第二个目标函数取最大值。

下面以单目标优化为例。

使用creator创建单目标优化问题（创建的是类对象，类名字是FitnessMin，继承Fitness，初始化参数是weights）后，还需要创建一个从列表派生的类Individual，其适应度属性设置为刚刚创建的适应度。

In [1]:
from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

## 个体编码

创建类型后，需要用值填充它们，值的形式有多种。比如实数编码(Value encoding)：直接用实数对变量进行编码。优点是不用解码，基因表达非常简洁，而且能对应连续区间。但是实数编码后搜索区间连续，因此容易陷入局部最优。

具体执行时，使用 Toolbox 工具（Toolbox是各种工具的容器）。用random.random来产生随机数，给到5个实数编码的个体。

In [2]:
from deap import tools
import random

IND_SIZE = 5
toolbox = base.Toolbox()
toolbox.register('Attr_float', random.random)
toolbox.register('Individual', tools.initRepeat, creator.Individual, toolbox.Attr_float, n=IND_SIZE)

ind1 = toolbox.Individual()
print(ind1)

[0.520649853072272, 0.5141796254186876, 0.8911134057159886, 0.1946932611778992, 0.9481968821161931]


还有其他编码方式，暂不一一举例。

## 初始族群的创建

接下来可以创建族群。

一般族群：这是最常用的族群类型，族群中没有特别的顺序或者子族群。

继续以实数编码为例，以下代码可以生成由10个长度为5的随机实数编码个体组成的一般族群：

In [3]:
N_POP = 10
toolbox.register('Population', tools.initRepeat, list, toolbox.Individual)
pop = toolbox.Population(n = N_POP)

In [4]:
print(pop)

[[0.7231993197890395, 0.5714906434212933, 0.6055336248695616, 0.05488529996278568, 0.8064326449145963], [0.6899802091678227, 0.6515772887360454, 0.6237934723671826, 0.9554487452537622, 0.8218987596963334], [0.032355872378507455, 0.3650535158809399, 0.7839416032487232, 0.4712413880420072, 0.9387558115353909], [0.6307084138234457, 0.17558027545368382, 0.043822531147390764, 0.9005425662770163, 0.8371530733196973], [0.08457092428679824, 0.42619519256428073, 0.7838978354616313, 0.9336699304248721, 0.3167161494182369], [0.9063545945206749, 0.4965295579978829, 0.46568065496374955, 0.8814326205202351, 0.8771680989819435], [0.27567721719357674, 0.1369181571005299, 0.9611366206378926, 0.5205024293413351, 0.5350485431263744], [0.17776830381796127, 0.28238916108684964, 0.4744047605215007, 0.19572919842469216, 0.13047333021666518], [0.6251586727513625, 0.5500451366380162, 0.08294430904739836, 0.7738953071846995, 0.3708678113653123], [0.9168085825469837, 0.08443743218636335, 0.5934586866121827, 0.97

## 评价

评价部分是根据任务的特性高度定制的，DEAP库中并没有预置的评价函数模版。

在使用DEAP时，需要注意的是，无论是单目标还是多目标优化，评价函数的返回值必须是一个tuple类型。

In [5]:
# 定义评价函数
def evaluate(individual):
    return sum(individual), #注意这个逗号，即使是单变量优化问题，也需要返回tuple

# 评价初始族群
toolbox.register('Evaluate', evaluate)
fitnesses = map(toolbox.Evaluate, pop)
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit
    print(ind.fitness.values)

(2.761541532957277,)
(3.7426984752211463,)
(2.5913481910855687,)
(2.587806860021234,)
(2.5450500321558196,)
(3.627165526984486,)
(2.4292829673997085,)
(1.260764754067669,)
(2.402911236986789,)
(3.104777282865403,)


## 配种选择

DEAP的tools模块中内置了13种选择操作，对全部选择算子的描述可以参考[官方文档](https://deap.readthedocs.io/en/master/api/tools.html#deap.tools.selTournament)。

|函数|	简介|
|-|-|
|selTournament()|	锦标赛选择|
|selRoulette()|	轮盘赌选择（不能用于最小化或者适应度会小于等于0的问题）|
|selNSGA2()|	NSGA-II选择，适用于多目标遗传算法|
|selSPEA2()|	SPEA2选择，目前版本(ver 1.2.2)的该函数实现有误，没有为个体分配距离，不建议使用。|
|selRandom()|	有放回的随机选择|
|selBest()|	选择最佳|
|selWorst()|	选择最差|
|selTournamentDCD()|	Dominance/Crowding distance锦标赛选择，目前版本的实现也有些问题|
|selDoubleTournament()|	Size+Fitness双锦标赛选择|
|selStochasticUniversalSampling()|	随机抽样选择|
|selLexicase()|	词典选择|
|selEpsilonLexicase()|	词典选择在连续值域上的扩展|
|selAutomaticEpsilonLexicase()|	|

比如锦标赛选择：deap.tools.selTournament(individuals, k, tournsize, fit_attr = 'fitness')

就是模拟锦标赛的方式，首先在族群中随机抽取tournsize个个体，然后从中选取具有最佳适应度的个体，将此过程重复k次，获得育种族群。tournsize越大，选择强度(selection intensity)越高，在选择之后留下的育种族群的平均适应度也就越高。

下图给出了由5个个体构成的族群中进行一次tournsize为3的锦标赛选择的过程。

![](pictures/17867674-d94652a0f18353b2.webp)

还有轮盘赌选择，它是最常见的选择策略，可以看作是有放回的随机抽样。

轮盘赌选择: deap.tools.selRoulette(individuals, k, fit_attr = 'fitness')

在轮盘赌选择中，每个个体$a_i$被选中的概率$P(a_i)$与其适应度函数$f(a_i)$成正比：

$$P(a_i)=\frac{f(a_i)}{\sum _{i} f(a_i)}$$

下图给出了与前文同样例子的轮盘赌选择

![](pictures/17867674-3c503fc666584cf5.webp)

但在实际应用中，很多文章都指出轮盘赌选择的性能较差，在通常情况下都不如随机抽样选择和锦标赛选择。

随机普遍抽样选择：deap.tools.selStochasticUniversalSampling(individuals, k, fit_attr = 'fitness')

随机普遍抽样选择是一种有多个指针的轮盘赌选择，其优点是能够保存族群多样性，而不会像轮盘赌一样，有较大几率对重复选择最优个体。

在与前文相同的例子中使用随机普遍抽样选择，设定指针数k为3，那么指针间距即为1/3

![](pictures/17867674-c20b83385c700292.webp)

下面以锦标赛为例，给出代码示例。

In [6]:
# 选择方式：锦标赛选择
toolbox.register('TourSel', tools.selTournament, tournsize = 2) # 注册Tournsize为2的锦标赛选择
selectedTour = toolbox.TourSel(pop, 5) # 选择5个个体
print('锦标赛选择结果：')
for ind in selectedTour:
    print(ind)
    print(ind.fitness.values)

锦标赛选择结果：
[0.9168085825469837, 0.08443743218636335, 0.5934586866121827, 0.9712997759213877, 0.5387728055984855]
(3.104777282865403,)
[0.27567721719357674, 0.1369181571005299, 0.9611366206378926, 0.5205024293413351, 0.5350485431263744]
(2.4292829673997085,)
[0.9063545945206749, 0.4965295579978829, 0.46568065496374955, 0.8814326205202351, 0.8771680989819435]
(3.627165526984486,)
[0.6251586727513625, 0.5500451366380162, 0.08294430904739836, 0.7738953071846995, 0.3708678113653123]
(2.402911236986789,)
[0.6251586727513625, 0.5500451366380162, 0.08294430904739836, 0.7738953071846995, 0.3708678113653123]
(2.402911236986789,)


## 变异

变异过程就是从父代的基因出发，进行操作，最终得到子代基因的过程。通常包括交叉(Crossover)和突变(Mutation)两种操作。

### 交叉

DEAP内置的交叉(Crossover)操作

|函数|	简介|	适用编码方式|
|-|-|-|
|cxOnePoint()|	单点交叉|	实数、二进制|
|cxTwoPoint()|	两点交叉|	实数、二进制|
|cxUniform()|	均匀交叉|	实数、二进制|
|cxPartialyMatched()|	部分匹配交叉PMX|	序列|
|cxUniformPartialyMatched()|	PMX变种，改两点为均匀交叉|	序列|
|cxOrdered()|	有序交叉|	序列|
|cxBlend()|	混合交叉|	实数|
|cxESBlend()|	带进化策略的混合交叉	||
|cxESTwoPoint()|	带进化策略的两点交叉|	|
|cxSimulatedBinary()|	模拟二值交叉|	实数|
|cxSimulatedBinaryBounded()|	有界模拟二值交叉|	实数|
|cxMessyOnePoint()|	混乱单点交叉|	实数、二进制|

比如单点交叉：deap.tools.cxOnePoint(ind1, ind2)

最简单的交叉方式，选择一个切口，将两条基因切开之后，交换尾部基因段。该方法非常简单，但是多篇文章指出，该算法在各种实验中性能都被其他交叉算法吊打，因此算是一种不建议使用的loser algorithm。

两点交叉：deap.tools.cxTwoPoint(ind1, ind2)

用两个点切开基因之后，交换切出来的基因段。

![](pictures/17867674-b08f2a38bd9540c7.webp)

比如均匀交叉：deap.tools.cxUniform(ind1, ind2, indpb)

指定一个变异几率，两父代中的每个基因都以该几率交叉。

![](pictures/17867674-61e22658b6147854.webp)

官方提示最好不要直接用父代进行交叉，因为有些交叉算法是in-place运算的，因此最好先复制，再进行交叉。具体见代码：

In [7]:
ind1, ind2 = [toolbox.Individual() for _ in range(2)]
print(ind1, '\n', ind2)

[0.9831912647683978, 0.22964078049870063, 0.9646927551069919, 0.10529854091151258, 0.834897003879722] 
 [0.21145040742451904, 0.7280997352983748, 0.46201856039087763, 0.44028249222698623, 4.4507172898389236e-05]


In [8]:
# 均匀交叉
child1, child2 = [toolbox.clone(ind) for ind in (ind1, ind2)]
tools.cxUniform(child1, child2, 0.5)
print(child1, '\n', child2)

[0.9831912647683978, 0.7280997352983748, 0.46201856039087763, 0.10529854091151258, 0.834897003879722] 
 [0.21145040742451904, 0.22964078049870063, 0.9646927551069919, 0.44028249222698623, 4.4507172898389236e-05]


### 突变

DEAP内置的突变(Mutation)操作：

|函数|	简介|	适用编码方式|
|-|-|-|
|mutGaussian()|	高斯突变|	实数|
|mutShuffleIndexes()|	乱序突变|	实数、二进制、序列|
|mutFlipBit()|	位翻转突变|	二进制|
|mutPolynomialBounded()|	有界多项式突变	|实数|
|mutUniformInt()|	均匀整数突变	|实数、序列|
|mutESLogNormal()		||

比如高斯突变：tools.mutGaussian(individual, mu, sigma, indpb)

对个体序列中的每一个基因按概率变异，变异后的值为按均值为，方差为的高斯分布选取的一个随机数。如果不希望均值发生变化，则应该将设为0。

和交叉选择相同，如果想要保留父代作为参照，那么最好先复制，然后再进行变异：

In [9]:
mutant = toolbox.clone(ind1)
tools.mutGaussian(mutant, 3, 0.1, 1)
print(mutant)
# 可以看到当均值给到3之后，变异形成的个体均值从0.5也增大到了3附近

[4.0945855547703776, 3.3644804469270744, 3.8402254771138824, 3.1387477364461356, 3.666694870938168]


## 环境选择

环境选择也就是重插入(Reinsertion)，在选择、交叉和突变之后，得到的育种后代族群规模与父代相比可能增加或减少。为保持族群规模，需要将育种后代插入到父代中，替换父代种群的一部分个体，或者丢弃一部分育种个体。

重插入分为全局重插入(Global reinsertion)和本地重插入(Local reinsertion)两种，后者只有在使用含有本地邻域的算法时使用。常见的全局重插入操作有以下四种：

- 完全重插入(Pure reinsertion)：产生与父代个体数量相当的配种个体，直接用配种个体生成新一代族群。
- 均匀重插入(Uniform reinsertion)：产生比父代个体少的配种个体，用配种个体随机均匀地替换父代个体。
- 精英重插入(Elitist reinsertion)：产生比父代个体少的配种个体，选取配种后代中适应度最好的一些个体，插入父代中，取代适应度较低的父代个体。
- 精英保留重插入(Fitness-based reinsertion)：产生比父代个体多的配种个体，选取其中适应度最大的配种个体形成新一代族群。

通常来说后两种方式由于精英保留的缘故，收敛速度较快，因此比较推荐。

DEAP中没有设定专门的reinsertion操作，可以用选择操作选择中的selBest, selWorst,selRandom来对育种族群和父代族群进行操作。

下面是一个相对完整的例子。

In [2]:
from deap import base, creator
import random
from deap import tools

In [3]:
# 前面已经创建过这些类了，所以这里不重复了，不过单独作一个完整例子时，这些必不可少
# creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# creator.create("Individual", list, fitness=creator.FitnessMin)

In [4]:
IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [5]:
def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [6]:
pop = toolbox.population(n=50)
CXPB, MUTPB, NGEN = 0.5, 0.2, 40

# Evaluate the entire population
fitnesses = map(toolbox.evaluate, pop)
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

for g in range(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring

更多例子可以参考[这里](https://deap.readthedocs.io/en/master/examples/ga_onemax.html)

还有一些常见的问题，比如

- 多参数设定，可以参考：[Optimization with multiple variables #304](https://github.com/DEAP/deap/issues/304)
- 评价函数多参数，可以参考：[How to write loss function with extra parameters? #331](https://github.com/DEAP/deap/issues/331)